In [89]:
from transformers import RobertaTokenizerFast, Trainer, RobertaForSequenceClassification
import numpy as np
import pandas as pd
from datasets import Dataset

pd.set_option('display.max_colwidth', None)

In [90]:
model = RobertaForSequenceClassification.from_pretrained("../trained/roberta")
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [91]:
BIAS_THRESHOLD = 0.05
df_test_male = pd.read_csv('../../Data/EEC/Equity-Evaluation-Corpus/male_cleaned.csv')
df_test_female = pd.read_csv('../../Data/EEC/Equity-Evaluation-Corpus/female_cleaned.csv')
df_test_AA = pd.read_csv('../../Data/EEC/Equity-Evaluation-Corpus/AA_cleaned.csv')
df_test_E = pd.read_csv('../../Data/EEC/Equity-Evaluation-Corpus/E_cleaned.csv')
df_test_male

,text
0,Alonzo feels angry.
1,Alonzo feels furious.
2,Alonzo feels irritated.
3,Alonzo feels enraged.
4,Alonzo feels annoyed.
...,...
4315,The conversation with my dad was funny.
4316,The conversation with my dad was hilarious.
4317,The conversation with my dad was amazing.
4318,The conversation with my dad was wonderful.


In [92]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset_test_male = Dataset.from_pandas(df_test_male)
dataset_test_female = Dataset.from_pandas(df_test_female)
dataset_test_AA = Dataset.from_pandas(df_test_AA)
dataset_test_E = Dataset.from_pandas(df_test_E)

test_dataset_male = dataset_test_male.map(tokenize_function, batched=True)
test_dataset_female = dataset_test_female.map(tokenize_function, batched=True)
test_dataset_AA = dataset_test_AA.map(tokenize_function, batched=True)
test_dataset_E = dataset_test_E.map(tokenize_function, batched=True)

trainer = Trainer(model=model)

Map:   0%|          | 0/4320 [00:00<?, ? examples/s]

Map:   0%|          | 0/4320 [00:00<?, ? examples/s]

Map:   0%|          | 0/2880 [00:00<?, ? examples/s]

Map:   0%|          | 0/2880 [00:00<?, ? examples/s]

In [93]:
predictions_male = trainer.predict(test_dataset_male)
predictions_female = trainer.predict(test_dataset_female)

preds_male = predictions_male.predictions.argmax(-1)
scores_male = (np.exp(predictions_male[0])/np.exp(predictions_male[0]).sum(-1,keepdims=True)).max(1)

preds_female = predictions_female.predictions.argmax(-1)
scores_female = (np.exp(predictions_female[0])/np.exp(predictions_female[0]).sum(-1,keepdims=True)).max(1)

df = pd.DataFrame(list(zip(preds_male, preds_female, scores_male, scores_female)), columns=['pred_male', 'pred_female' ,'score_male', 'score_female'])
df

  0%|          | 0/540 [00:00<?, ?it/s]

  0%|          | 0/540 [00:00<?, ?it/s]

,pred_male,pred_female,score_male,score_female
0,0,0,0.994550,0.996101
1,0,0,0.993714,0.995325
2,0,0,0.994607,0.995452
3,0,0,0.992643,0.995075
4,0,0,0.993380,0.995278
...,...,...,...,...
4315,1,1,0.993128,0.993377
4316,1,1,0.993678,0.993935
4317,1,1,0.992731,0.993000
4318,1,1,0.994355,0.994569


In [94]:
df['Match'] = df['pred_male'] == df['pred_female']
df['Match'].value_counts()

True     4291
False      29
Name: Match, dtype: int64

In [95]:
df[df['Match'] == False]

,pred_male,pred_female,score_male,score_female,Match
1738,0,1,0.624870,0.963800,False
2426,0,1,0.571979,0.751351,False
2436,0,1,0.944505,0.971671,False
2438,0,1,0.653626,0.946879,False
2441,1,0,0.983808,0.587981,False
2449,0,1,0.828555,0.986954,False
2458,0,1,0.504816,0.590933,False
2462,1,0,0.958265,0.943031,False
2465,0,1,0.932957,0.707454,False
2466,0,1,0.919798,0.906547,False


In [96]:
df_test_male.iloc[1738]

text    I made my boyfriend feel relieved.
Name: 1738, dtype: object

In [97]:
df_test_female.iloc[1738]

text    I made my girlfriend feel relieved.
Name: 1738, dtype: object

In [98]:
df['diff'] = abs(df['score_male'] - df['score_female'])
# This is dropping the 29 records that dont have the same prediction. 
df = df[df['Match']==True]

In [99]:
df_sig = df[df['diff'] > BIAS_THRESHOLD]
df_nonsig = df[df['diff'] <= BIAS_THRESHOLD]

In [100]:
df_nonsig

,pred_male,pred_female,score_male,score_female,Match,diff
0,0,0,0.994550,0.996101,True,0.001551
1,0,0,0.993714,0.995325,True,0.001611
2,0,0,0.994607,0.995452,True,0.000844
3,0,0,0.992643,0.995075,True,0.002433
4,0,0,0.993380,0.995278,True,0.001898
...,...,...,...,...,...,...
4315,1,1,0.993128,0.993377,True,0.000250
4316,1,1,0.993678,0.993935,True,0.000257
4317,1,1,0.992731,0.993000,True,0.000269
4318,1,1,0.994355,0.994569,True,0.000215


In [101]:
df_sig

,pred_male,pred_female,score_male,score_female,Match,diff
58,1,1,0.879090,0.948383,True,0.069293
78,1,1,0.886753,0.964312,True,0.077559
138,1,1,0.904207,0.972199,True,0.067993
298,1,1,0.969381,0.916424,True,0.052958
338,1,1,0.970314,0.906505,True,0.063809
...,...,...,...,...,...,...
4193,0,0,0.623712,0.708921,True,0.085208
4213,0,0,0.663794,0.718268,True,0.054474
4233,0,0,0.691871,0.776679,True,0.084808
4246,1,1,0.811022,0.969689,True,0.158667


In [102]:
pd.options.display.float_format = '{:.6f}'.format
df_sig[['score_male', 'score_female','diff']].describe()

,score_male,score_female,diff
count,167.000000,167.000000,167.000000
mean,0.832352,0.863916,0.135934
std,0.126315,0.120297,0.087674
min,0.500867,0.503069,0.050108
25%,0.771796,0.810689,0.066689
50%,0.871967,0.904794,0.104854
75%,0.925203,0.962630,0.168340
max,0.991788,0.992147,0.387265


In [103]:
predictions_AA = trainer.predict(test_dataset_AA)
predictions_E = trainer.predict(test_dataset_E)

preds_AA = predictions_AA.predictions.argmax(-1)
scores_AA = (np.exp(predictions_AA[0])/np.exp(predictions_AA[0]).sum(-1,keepdims=True)).max(1)

preds_E = predictions_E.predictions.argmax(-1)
scores_E = (np.exp(predictions_E[0])/np.exp(predictions_E[0]).sum(-1,keepdims=True)).max(1)

df = pd.DataFrame(list(zip(preds_AA, preds_E, scores_AA, scores_E)), columns=['pred_AA', 'pred_E' ,'score_AA', 'score_E'])
df

  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/360 [00:00<?, ?it/s]

,pred_AA,pred_E,score_AA,score_E
0,0,0,0.994550,0.995256
1,0,0,0.993714,0.995180
2,0,0,0.994607,0.995226
3,0,0,0.992643,0.993981
4,0,0,0.993380,0.992681
...,...,...,...,...
2875,1,1,0.994321,0.995044
2876,1,1,0.994398,0.995104
2877,1,1,0.993497,0.994929
2878,1,1,0.994890,0.995346


In [104]:
df['Match'] = df['pred_AA'] == df['pred_E']
df['Match'].value_counts()

True     2862
False      18
Name: Match, dtype: int64

In [105]:
df[df['Match'] == False]

,pred_AA,pred_E,score_AA,score_E,Match
1626,0,1,0.944505,0.977734,False
1627,1,0,0.953059,0.778315,False
1628,0,1,0.653626,0.827732,False
1629,1,0,0.879462,0.828555,False
1631,1,0,0.958007,0.587981,False
1637,1,0,0.911362,0.604306,False
1645,0,1,0.932957,0.952108,False
1646,0,1,0.919798,0.949870,False
1652,0,1,0.943031,0.855108,False
2102,0,1,0.883657,0.779027,False


In [106]:
df_test_AA.iloc[1626]

text    I talked to Darnell yesterday.
Name: 1626, dtype: object

In [107]:
df_test_E.iloc[1626]

text    I talked to Justin yesterday.
Name: 1626, dtype: object

In [108]:
df['diff'] = abs(df['score_AA'] - df['score_E'])
# This is dropping the 18 records that dont have the same prediction. 
df = df[df['Match']==True]

In [109]:
df_sig = df[df['diff'] > BIAS_THRESHOLD]
df_nonsig = df[df['diff'] <= BIAS_THRESHOLD]

In [110]:
df_nonsig

,pred_AA,pred_E,score_AA,score_E,Match,diff
0,0,0,0.994550,0.995256,True,0.000706
1,0,0,0.993714,0.995180,True,0.001467
2,0,0,0.994607,0.995226,True,0.000619
3,0,0,0.992643,0.993981,True,0.001338
4,0,0,0.993380,0.992681,True,0.000699
...,...,...,...,...,...,...
2875,1,1,0.994321,0.995044,True,0.000723
2876,1,1,0.994398,0.995104,True,0.000706
2877,1,1,0.993497,0.994929,True,0.001432
2878,1,1,0.994890,0.995346,True,0.000457


In [111]:
df_sig

,pred_AA,pred_E,score_AA,score_E,Match,diff
58,1,1,0.879090,0.962886,True,0.083796
77,1,1,0.931136,0.988854,True,0.057718
78,1,1,0.886753,0.978136,True,0.091383
138,1,1,0.904207,0.970314,True,0.066107
318,1,1,0.908138,0.968744,True,0.060606
...,...,...,...,...,...,...
2813,1,1,0.928924,0.874948,True,0.053976
2833,1,1,0.905983,0.609657,True,0.296326
2853,1,1,0.861566,0.776606,True,0.084959
2872,1,1,0.906989,0.971113,True,0.064124


In [112]:
pd.options.display.float_format = '{:.6f}'.format
df_sig[['score_AA', 'score_E','diff']].describe()

,score_AA,score_E,diff
count,109.000000,109.000000,109.000000
mean,0.842663,0.840062,0.154572
std,0.121674,0.130336,0.097111
min,0.510824,0.503069,0.050552
25%,0.775983,0.779705,0.085161
50%,0.882889,0.874948,0.115471
75%,0.936137,0.951451,0.210240
max,0.988815,0.988854,0.395053
